# AST 4930 Module 3

## Support vector machines

### Let's load the scikit-learn module and load the Iris dataset.

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()

### Let's visualize the data. Petal width vs. Petal length

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6, 4))

norm = plt.Normalize(vmin=iris.target.min(), vmax=iris.target.max())

setosa = ax.scatter(iris.data[:,2][iris.target == 1], iris.data[:,3][iris.target == 1], 
                    alpha=0.7, c=iris.target[iris.target == 1], cmap='viridis', norm=norm)

versicolor = ax.scatter(iris.data[:,2][iris.target == 2], iris.data[:,3][iris.target == 2], 
                        alpha=0.7, c=iris.target[iris.target == 2], cmap='viridis', norm=norm)

ax.set_xlim(2.5,7.5)
ax.set_ylim(0.8,2.7)
ax.set_xlabel(iris.feature_names[2])
ax.set_ylabel(iris.feature_names[3])
ax.legend([setosa, versicolor], iris.target_names[0:2], loc='upper left')

plt.savefig('iris.png',dpi=300)

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split

# Using petal width/length only.
X = iris.data[:,2:]
y = (iris.target == 2).astype(np.float64)


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# We need feature scaling for SVM.
scaler = StandardScaler()

# Here I'm making three linear SVM Classification models with three different C values.
svm_clf1 = LinearSVC(C=1, random_state=0)
svm_clf2 = LinearSVC(C=100, random_state=0)
svm_clf3 = LinearSVC(C=0.1, random_state=0)


# Making pipelines.
scaled_svm_clf1 = Pipeline([
        ("scaler", scaler),
        ("linear_svc", svm_clf1),
    ])
scaled_svm_clf2 = Pipeline([
        ("scaler", scaler),
        ("linear_svc", svm_clf2),
    ])
scaled_svm_clf3 = Pipeline([
        ("scaler", scaler),
        ("linear_svc", svm_clf3),
    ])

# Fit the data.
scaled_svm_clf1.fit(X_train, y_train)
scaled_svm_clf2.fit(X_train, y_train)
scaled_svm_clf3.fit(X_train, y_train)

In [ ]:
print(scaled_svm_clf1.score(X_test, y_test),
      scaled_svm_clf2.score(X_test, y_test),
      scaled_svm_clf3.score(X_test, y_test))

In [ ]:
# This cell is to visualize the support vectors and margins.

# Convert to unscaled parameters
b1 = svm_clf1.decision_function([-scaler.mean_ / scaler.scale_])
b2 = svm_clf2.decision_function([-scaler.mean_ / scaler.scale_])
b3 = svm_clf3.decision_function([-scaler.mean_ / scaler.scale_])

w1 = svm_clf1.coef_[0] / scaler.scale_
w2 = svm_clf2.coef_[0] / scaler.scale_
w3 = svm_clf3.coef_[0] / scaler.scale_

svm_clf1.intercept_ = np.array([b1])
svm_clf2.intercept_ = np.array([b2])
svm_clf3.intercept_ = np.array([b3])

svm_clf1.coef_ = np.array([w1])
svm_clf2.coef_ = np.array([w2])
svm_clf3.coef_ = np.array([w3])

# Find support vectors (LinearSVC does not do this automatically). 
t = y * 2 - 1
support_vectors_idx1 = (t * (X.dot(w1) + b1) < 1).ravel()
support_vectors_idx2 = (t * (X.dot(w2) + b2) < 1).ravel()
support_vectors_idx3 = (t * (X.dot(w3) + b3) < 1).ravel()

svm_clf1.support_vectors_ = X[support_vectors_idx1]
svm_clf2.support_vectors_ = X[support_vectors_idx2]
svm_clf3.support_vectors_ = X[support_vectors_idx3]

In [ ]:
def plot_svc_decision_boundary(svm_clf, xmin, xmax):
    w = svm_clf.coef_[0]
    b = svm_clf.intercept_[0]

    # At the decision boundary, w0*x0 + w1*x1 + b = 0
    # => x1 = -w0/w1 * x0 - b/w1
    x0 = np.linspace(xmin, xmax, 200)
    decision_boundary = -w[0]/w[1] * x0 - b/w[1]

    margin = 1/w[1]
    gutter_up = decision_boundary + margin
    gutter_down = decision_boundary - margin

    svs = svm_clf.support_vectors_
    plt.scatter(svs[:, 0], svs[:, 1], s=120, edgecolors='black', facecolors='none', alpha=0.5)
    plt.plot(x0, decision_boundary, "k-", linewidth=2)
    plt.plot(x0, gutter_up, "k--", linewidth=2)
    plt.plot(x0, gutter_down, "k--", linewidth=2)

### Make a plot for the model with C=1

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

norm = plt.Normalize(vmin=iris.target.min(), vmax=iris.target.max())

setosa = ax.scatter(iris.data[:,2][iris.target == 1], iris.data[:,3][iris.target == 1], 
                    alpha=0.7, c=iris.target[iris.target == 1], cmap='viridis', norm=norm)

versicolor = ax.scatter(iris.data[:,2][iris.target == 2], iris.data[:,3][iris.target == 2], 
                        alpha=0.7, c=iris.target[iris.target == 2], cmap='viridis', norm=norm)

plot_svc_decision_boundary(svm_clf1, 2.5, 7.5)

ax.set_xlim(2.5,7.5)
ax.set_ylim(0.8,2.7)
ax.set_xlabel(iris.feature_names[2])
ax.set_ylabel(iris.feature_names[3])
ax.legend([setosa, versicolor], iris.target_names[0:2], loc='upper left')

plt.savefig('iris_SVM_C1.png',dpi=300)

### Make a plot for the model with C=100

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

norm = plt.Normalize(vmin=iris.target.min(), vmax=iris.target.max())

setosa = ax.scatter(iris.data[:,2][iris.target == 1], iris.data[:,3][iris.target == 1], 
                    alpha=0.7, c=iris.target[iris.target == 1], cmap='viridis', norm=norm)

versicolor = ax.scatter(iris.data[:,2][iris.target == 2], iris.data[:,3][iris.target == 2], 
                        alpha=0.7, c=iris.target[iris.target == 2], cmap='viridis', norm=norm)

plot_svc_decision_boundary(svm_clf2, 2.5, 7.5)


ax.set_xlim(2.5,7.5)
ax.set_ylim(0.8,2.7)
ax.set_xlabel(iris.feature_names[2])
ax.set_ylabel(iris.feature_names[3])
ax.legend([setosa, versicolor], iris.target_names[0:2], loc='upper left')
plt.savefig('iris_SVM_C100.png',dpi=300)

### Make a plot for the model with C=0.1

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

norm = plt.Normalize(vmin=iris.target.min(), vmax=iris.target.max())

setosa = ax.scatter(iris.data[:,2][iris.target == 1], iris.data[:,3][iris.target == 1], 
                    alpha=0.7, c=iris.target[iris.target == 1], cmap='viridis', norm=norm)

versicolor = ax.scatter(iris.data[:,2][iris.target == 2], iris.data[:,3][iris.target == 2], 
                        alpha=0.7, c=iris.target[iris.target == 2], cmap='viridis', norm=norm)

plot_svc_decision_boundary(svm_clf3, 2.5, 7.5)


ax.set_xlim(2.5,7.5)
ax.set_ylim(0.8,2.7)
ax.set_xlabel(iris.feature_names[2])
ax.set_ylabel(iris.feature_names[3])
ax.legend([setosa, versicolor], iris.target_names[0:2], loc='upper left')
plt.savefig('iris_SVM_C01.png',dpi=300)

### Multi-class classification with SVM

In [ ]:
from sklearn import datasets
import numpy as np

iris = datasets.load_iris()

X = iris.data[:,2:]
y = iris.target

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC

model = Pipeline([
    ('sc', StandardScaler()),
    ('svm', SVC(C=1, kernel="linear", decision_function_shape='ovo', random_state=0))
])

model.fit(X, y)

### Let's compute the decision boundary.

In [ ]:
#Determine the minimum and maximum x & y ranges for the plot
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

print(x_min, x_max, y_min, y_max)

In [ ]:
x_min = np.max([x_min, 0.])
y_min = np.max([y_min, 0.])

print(x_min, x_max, y_min, y_max)

In [ ]:
xpts, ypts = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

Z = model.predict(np.c_[xpts.ravel(), ypts.ravel()])
Z = Z.reshape(xpts.shape)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6, 4))

ax.contourf(xpts, ypts, Z, alpha=0.5)

norm = plt.Normalize(vmin=iris.target.min(), vmax=iris.target.max())

#Let's over-plot training/test data points.
plot_train = ax.scatter(X[:,0], X[:,1], alpha=0.7, c=y, cmap='viridis', 
                        norm=norm, edgecolor='black')

ax.set_xlabel(iris.feature_names[2])
ax.set_ylabel(iris.feature_names[3])

### Looking good!

### Classification for non-linear data.

In [ ]:
import sklearn
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6, 6))

X,y = sklearn.datasets.make_circles(noise=0.1, factor=0.5, random_state=0)

plt.scatter(X[:, 0], X[:, 1], c=y)

ax.set_xlabel("Feature 1")
ax.set_ylabel("Feature 2")
ax.set_xlim(-1.4,1.4)
ax.set_ylim(-1.4,1.4)

### How do the data look when mapped onto a 3D space?

In [ ]:
import numpy as np

# add the squared first feature
X_new = np.hstack([X, X[:,0:1]**2 + X[:, 1:2] ** 2])

from mpl_toolkits.mplot3d import Axes3D, axes3d

fig = plt.figure()

# visualize in 3D
ax = fig.add_subplot(projection='3d')

# plot first all the points with y==0, then all with y == 1
mask = y == 0
ax.scatter(X_new[mask, 0], X_new[mask, 1], X_new[mask, 2], 
           s=60, edgecolor='k')
ax.scatter(X_new[~mask, 0], X_new[~mask, 1], X_new[~mask, 2], marker='^',
           s=60, edgecolor='k')

ax.set_xlim(-1.4,1.4)
ax.set_ylim(-1.4,1.4)
ax.set_xlabel("feature1")
ax.set_ylabel("feature2")
ax.set_zlabel("feature1**2 + feature2**2")


### Adding the hyperplane.

In [ ]:
from sklearn.svm import LinearSVC

# Here I'm using a linear version of SVC.
linear_svm_3d = LinearSVC().fit(X_new, y)
coef, intercept = linear_svm_3d.coef_.ravel(), linear_svm_3d.intercept_

# Show linear decision boundary.
fig = plt.figure()

# visualize in 3D
ax = fig.add_subplot(projection='3d')

xx = np.linspace(X_new[:, 0].min() - 0.5, X_new[:, 0].max() + 0.5, 100)
yy = np.linspace(X_new[:, 1].min() - 0.5, X_new[:, 1].max() + 0.5, 100)

XX, YY = np.meshgrid(xx, yy)
ZZ = (coef[0] * XX + coef[1] * YY + intercept) / -coef[2]

ax.plot_surface(XX, YY, ZZ, rstride=8, cstride=8, alpha=0.3, color='gray')
ax.scatter(X_new[mask, 0], X_new[mask, 1], X_new[mask, 2], 
           cmap=mglearn.cm2, s=60, edgecolor='k')
ax.scatter(X_new[~mask, 0], X_new[~mask, 1], X_new[~mask, 2], marker='^',
           cmap=mglearn.cm2, s=60, edgecolor='k')

ax.set_xlim(-1.4,1.4)
ax.set_ylim(-1.4,1.4)
ax.set_xlabel("feature1")
ax.set_ylabel("feature2")
ax.set_zlabel("feature1**2 + feature2**2")

### This is how the hyperplane looks like in the original 2D space.

In [ ]:
ZZ = XX**2 + YY**2
dec = linear_svm_3d.decision_function(np.c_[XX.ravel(), YY.ravel(), ZZ.ravel()])

fig, ax = plt.subplots(figsize=(6, 6))

ax.contourf(XX, YY, dec.reshape(XX.shape), levels=[dec.min(), 0, dec.max()],
             cmap=mglearn.cm2, alpha=0.5)

mglearn.discrete_scatter(X[:, 0], X[:, 1], y)

ax.set_xlabel("Feature 1")
ax.set_ylabel("Feature 2")
ax.set_xlim(-1.4,1.4)
ax.set_ylim(-1.4,1.4)

### How do kNN and DT perform on the same problem?

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Here's kNN.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors': np.arange(10)+1,
              'weights': ['uniform','distance']}

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, return_train_score=True, 
                           verbose=1)
grid_search.fit(X_train, y_train)

print("Best parameters: {}".format(grid_search.best_params_))
print("Best model: {}".format(grid_search.best_estimator_))
print("Test score: {:.2f}".format(grid_search.score(X_test, y_test)))


In [ ]:
xx = np.linspace(X_new[:, 0].min() - 0.5, X_new[:, 0].max() + 0.5, 100)
yy = np.linspace(X_new[:, 1].min() - 0.5, X_new[:, 1].max() + 0.5, 100)

XX, YY = np.meshgrid(xx, yy)

#This is to make a data structure that is consistent with the training/test datasets.
Z = grid_search.best_estimator_.predict(np.c_[XX.ravel(), YY.ravel()])

#Now let's reshape to match with the meshgrid.
Z = Z.reshape(XX.shape)

fig, ax = plt.subplots(figsize=(6, 6))

ax.contourf(XX, YY, Z, alpha=0.5, cmap=mglearn.cm2)

#Let's over-plot training/test data points.
mglearn.discrete_scatter(X[:, 0], X[:, 1], y)

ax.set_xlabel('feature 1')
ax.set_ylabel('feature 2')
ax.set_xlim(-1.4,1.4)
ax.set_ylim(-1.4,1.4)

### Here's DT.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': np.arange(10)+1,
              'criterion': ['gini','entropy']}

# By defaults, sklearn's GridSearchCV will use stratified k-fold for classification problems.
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, return_train_score=True, 
                           verbose=1)
grid_search.fit(X_train, y_train)

print("Best parameters: {}".format(grid_search.best_params_))
print("Best model: {}".format(grid_search.best_estimator_))
print("Test score: {:.2f}".format(grid_search.score(X_test, y_test)))


In [ ]:
xx = np.linspace(X_new[:, 0].min() - 0.5, X_new[:, 0].max() + 0.5, 100)
yy = np.linspace(X_new[:, 1].min() - 0.5, X_new[:, 1].max() + 0.5, 100)

XX, YY = np.meshgrid(xx, yy)

#This is to make a data structure that is consistent with the training/test datasets.
Z = grid_search.best_estimator_.predict(np.c_[XX.ravel(), YY.ravel()])

#Now let's reshape to match with the meshgrid.
Z = Z.reshape(XX.shape)

fig, ax = plt.subplots(figsize=(6, 6))

ax.contourf(XX, YY, Z, alpha=0.5, cmap=mglearn.cm2)

#Let's over-plot training/test data points.
mglearn.discrete_scatter(X[:, 0], X[:, 1], y)

ax.set_xlabel('feature 1')
ax.set_ylabel('feature 2')
ax.set_xlim(-1.4,1.4)
ax.set_ylim(-1.4,1.4)

### Let's try SVM on another non-linear problem. This time two moons instead of two circles.

In [ ]:
import sklearn
import mglearn
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6, 6))

# This is what it looks like without noise.
X, y = sklearn.datasets.make_moons(n_samples=200, noise=0., random_state=0)

mglearn.discrete_scatter(X[:, 0], X[:, 1], y)

ax.set_xlabel("Feature 1")
ax.set_ylabel("Feature 2")

In [ ]:
import sklearn
import mglearn
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6, 6))

# Now I'm adding some noise.
X,y = sklearn.datasets.make_moons(n_samples=200, noise=0.3, random_state=0)

mglearn.discrete_scatter(X[:, 0], X[:, 1], y)

ax.set_xlabel("Feature 1")
ax.set_ylabel("Feature 2")

### TODO: Run GridSearchCV and find the best SVM model. We will use "rbf" kernel and vary hyperparameter 'C' and 'gamma'.

### TODO: Let's make a heatmap and see if we are converged with the hyperparameters.

### TODO: Using the best model, plot the decision boundary.

### What is your best hyperpameters? What is your test score? How does the decision boundary look like?

### How do kNN and DT do on this problem? Choose kNN or DT and complete the cells below.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors': np.arange(30)+1,
              'weights': ['uniform','distance']}

# By defaults, sklearn's GridSearchCV will use stratified k-fold for classification problems.
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, return_train_score=True, 
                           verbose=1)
grid_search.fit(X_train, y_train)

print("Best parameters: {}".format(grid_search.best_params_))
print("Best model: {}".format(grid_search.best_estimator_))
print("Test score: {:.2f}".format(grid_search.score(X_test, y_test)))


### Let's plot the decision boundary.

In [ ]:
xx = np.linspace(X[:, 0].min() - 0.5, X[:, 0].max() + 0.5, 100)
yy = np.linspace(X[:, 1].min() - 0.5, X[:, 1].max() + 0.5, 100)

XX, YY = np.meshgrid(xx, yy)

#This is to make a data structure that is consistent with the training/test datasets.
Z = grid_search.best_estimator_.predict(np.c_[XX.ravel(), YY.ravel()])

#Now let's reshape to match with the meshgrid.
Z = Z.reshape(XX.shape)

fig, ax = plt.subplots(figsize=(6, 6))

ax.contourf(XX, YY, Z, alpha=0.5, cmap=mglearn.cm2)

#Let's over-plot training/test data points.
mglearn.discrete_scatter(X[:, 0], X[:, 1], y)

ax.set_xlabel('feature 1')
ax.set_ylabel('feature 2')

### Below is with DT.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': np.arange(30)+1,
              'criterion': ['gini','entropy']}

grid_search = GridSearchCV(DecisionTreeClassifier(random_state=0), param_grid, cv=5, return_train_score=True, 
                           verbose=1)
grid_search.fit(X_train, y_train)

print("Best parameters: {}".format(grid_search.best_params_))
print("Best model: {}".format(grid_search.best_estimator_))
print("Test score: {:.2f}".format(grid_search.score(X_test, y_test)))


### Let's plot the decision boundary.

In [ ]:
xx = np.linspace(X[:, 0].min() - 0.5, X[:, 0].max() + 0.5, 100)
yy = np.linspace(X[:, 1].min() - 0.5, X[:, 1].max() + 0.5, 100)

XX, YY = np.meshgrid(xx, yy)
Z = grid_search.best_estimator_.predict(np.c_[XX.ravel(), YY.ravel()])
Z = Z.reshape(XX.shape)

fig, ax = plt.subplots(figsize=(6, 6))

ax.contourf(XX, YY, Z, alpha=0.5, cmap=mglearn.cm2)

#Let's over-plot training/test data points.
mglearn.discrete_scatter(X[:, 0], X[:, 1], y)

ax.set_xlabel('feature 1')
ax.set_ylabel('feature 2')

## Permutation importance

### Let's load and split the Iris dataset.

In [ ]:
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()

X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Let's find the best SVM model.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Remember that we need feature scaling for SVM.
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('SVM', SVC(kernel='rbf'))
])

# Choose hyperparameters to optimize.
param_grid = {'SVM__C': [0.01, 0.1, 1., 10., 100.],
              'SVM__gamma': [0.01, 0.1, 1., 10., 100.]}

grid_search = GridSearchCV(pipe, param_grid, cv=5, return_train_score=True, verbose=1)
grid_search.fit(X_train, y_train)

print("Best parameters: {}".format(grid_search.best_params_))
print("Best model: {}".format(grid_search.best_estimator_))
print("Test score: {:.2f}".format(grid_search.score(X_test, y_test)))

### Let's compute the permutation importance.

In [ ]:
# We first fit our best model.

model = grid_search.best_estimator_[1]
model.fit(X_train, y_train)

# We then compute the permutation importance using the test data.
from sklearn.inspection import permutation_importance

r = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=0)

### Let's print out what's in the output.

In [ ]:
r

### Let's make a feature importance plot.

In [ ]:
import matplotlib.pyplot as plt

def plot_feature_importances(model):
    n_features = X.shape[1]
    plt.barh(np.arange(n_features), r.importances_mean, align='center')
    plt.yticks(np.arange(n_features), iris.feature_names)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)

plot_feature_importances(r)

### We can also make a box plot.

In [ ]:
sorted_idx = r.importances_mean.argsort()

fig, ax = plt.subplots()

ax.boxplot(r.importances[sorted_idx].T,
           vert=False, labels=np.array(iris.feature_names)[sorted_idx])

ax.set_xlabel("Permutation importance")
ax.set_ylabel("Feature")
